In [13]:
import os
import dotenv

dotenv.load_dotenv()
os.environ['GOOGLE_API_KEY'] = os.getenv('GOOGLE_API_KEY', 'your-key-is-not-using-env')

In [14]:
!pip install youtube-transcript-api==0.6.0 langchain-community langchain-google-genai faiss-cpu tiktoken python-dotenv google langchain

In [18]:
!pip install youtube-transcript-api

  Using cached youtube_transcript_api-1.2.3-py3-none-any.whl.metadata (24 kB)
Using cached youtube_transcript_api-1.2.3-py3-none-any.whl (485 kB)


## Import libraries

In [19]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_community.document_loaders import YoutubeLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_classic.chains import create_retrieval_chain
from langchain_classic.chains.combine_documents import create_stuff_documents_chain

In [32]:
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled, NoTranscriptFound

video_id = "CiPDEMxUfTY"

try:
    transcript_list = YouTubeTranscriptApi.list_transcripts(video_id)
    transcript = transcript_list.find_generated_transcript(['vi'])
    transcript_data = transcript.fetch()
    transcript_text = " ".join([x['text'] for x in transcript_data])

    print(transcript_text[:500] + "...")

except TranscriptsDisabled:
    print("Video không có phụ đề.")
except NoTranscriptFound:
    print("Không tìm thấy phụ đề tiếng Việt.")
except Exception as e:
    print("Lỗi khác:", e)


Lỗi khác: no element found: line 1, column 0


In [ ]:
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = splitter.create_documents([transcript_text])

In [ ]:
chunks

[Document(metadata={}, page_content='Vũ khí hiện đại rất quan trọng nhưng con người luôn là quyết định của mọi cuộc chiến. Mà hôm nay binh sĩ Campuchia cũng đã trở thành một ví dụ điển hình. Chào mừng các khán thính giả đã trở lại với Sao Đỏ TV trong bản tin chiến sự Campuchia Thái Lan ngày 15 tháng 12. Thưa các bạn, đúng là câu nói vũ khí quan trọng nhưng con người còn quan trọng hơn là không bao giờ sai. Vũ khí hiện đại rất tốt nhưng anh có khai thác sử dụng được hay không lại là chuyện khác. Hôm nay chúng ta sẽ tới với một ví dụ điển hình như vậy tại chiến trường Campuchia đại diện cho một sự kiện nổi bật vào ngày hôm qua trong cuộc xung đột này. Trước hết xin phép 1 phút điểm qua tình hình chiến sự sơ bộ. Đúng như tuyên bố không có chuyện ngừng bắn, ngay từ sáng sớm hôm qua, cả hai nước đã lao vào trận chiến ngay từ lúc bình minh. Khoảng 5:00 sáng ngày 14 tháng 12, quân đội Campuchia đã phát động các tấn công vào khu vực xung quanh đền Tà Kawai, đền Kho, Trông Gran, đồi 350, huyện 

## Embedding and turn them to vectorDB

In [ ]:
## Embedding
embeddings = GoogleGenerativeAIEmbeddings(
    model="models/text-embedding-004",
    google_api_key=os.environ["GOOGLE_API_KEY"]
)
vector_store = FAISS.from_documents(chunks, embeddings)

## Retrieve relevant documents

In [ ]:
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k":4})

retriever.invoke("Xe tăng được đề cập trong video là gì")

[Document(id='c748750b-a225-4714-8e29-23b2d3804e77', metadata={}, page_content='tên lửa qua liên kết dữ liệu để chọn mục tiêu chính xác vào phút chót. Tên lửa được trang bị động cơ kép gồm liều phóng lạnh đẩy đạn ra khỏi bệ trước khi động cơ chính kích hoạt ở cự ly an toàn so với xạ thủ vào bệ phóng. Qua đó cho phép phóng đạn từ hầm hào hoặc không gian kín tầm bán tối đao với phiên bản tiêu chuẩn là sáu cầy. Đặc biệt nếu được bổ sung thêm thiết bị dẫn hướng bổ trợ bao gồm ăten liên kết dữ liệu, màn hình điều khiển cầm tay và UAV tích hợp, tầm tấn công của tên lửa sẽ lên tới 10 cây. Với chế độ tăng tầm này, thường thì sẽ sử dụng chế độ bắn khóa sau khi phóng bởi lúc này tên lửa phóng sẽ không thấy được mục tiêu. Do đó, tên lửa sẽ được bắn về phía mục tiêu vốn đã được xác định vị trí tương đối. Trong quá trình bay, camera ở đầu tên lửa sẽ quay trực tiếp truyền về máy ngắm hay màn hình. Xạ thủ khi thấy mục tiêu trên màn hình thì kích vào khóa mục tiêu để tên lửa lao đến. Đây là phương phá

### Turn them to context for LLM

In [ ]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.3,
    google_api_key=os.environ["GOOGLE_API_KEY"]
)

NameError: name 'ChatGoogleGenerativeAI' is not defined

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "Bạn là trợ lý AI chuyên trả lời câu hỏi dựa trên nội dung video được cung cấp. "
    "Nếu thông tin không có trong context, hãy trả lời 'Không tìm thấy thông tin'. "
    "Trả lời ngắn gọn, súc tích trong 3 câu.\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}"),
])

question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)


query = "Xe tăng được đề cập trong video là gì?"

try:
    response = rag_chain.invoke({"input": query})
    
    print(f"Hỏi: {query}")
    print(f"Đáp: {response['answer']}")
    
    # Kiểm tra nguồn (đã rút gọn in ấn)
    print("\n--- Nguồn tham khảo ---")
    for i, doc in enumerate(response["context"]):
        print(f"[{i+1}] {doc.page_content[:100]}...")

except Exception as e:
    print(f"Lỗi: {e}")

NameError: name 'llm' is not defined